In [9]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
transformer = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True,
                                        transform=transformer, download=True)
testset = torchvision.datasets.CIFAR10(root="./data", train=False,
                                       transform=transformer, download=True)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

100%|██████████| 170M/170M [00:05<00:00, 29.1MB/s]


In [3]:
print(len(trainset))
print(len(testset))

50000
10000


In [4]:
trainset[0][0].shape # 3 цветовых канала, сами картинки размером 32 на 32 пикселя

torch.Size([3, 32, 32])

In [5]:
class MyOwnNet(nn.Module):
    def __init__(self):
      super(MyOwnNet, self).__init__()
      self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3)
      self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
      self.conv2 = nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3)
      self.conv3 = nn.Conv2d(in_channels=16, out_channels=24, kernel_size=3)

      self._to_linear = None
      self._get_conv_size(torch.randn(1, 3, 32, 32))

      self.flatten = nn.Flatten()
      self.fc1 = nn.Linear(self._to_linear, 120)
      self.fc2 = nn.Linear(120, 84)
      self.fc3 = nn.Linear(84, 10)

    def _get_conv_size(self, x):
          x = self.pool(F.relu(self.conv1(x)))
          x = self.pool(F.relu(self.conv2(x)))
          x = self.pool(F.relu(self.conv3(x)))
          self._to_linear = x.flatten(1).shape[1]

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # Последний слой без ФА, т.к. в Лосс уже встроен софтмакс
        return x

In [22]:
model = MyOwnNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [23]:
dataloaders = {"train": trainloader, "test": testloader}

In [24]:
model.train()
epochs = 100
accuracy = {"train": [], "test": []}
for epoch in tqdm(range(epochs)):
  for action, dataloader in dataloaders.items():
    epoch_correct = 0
    epoch_all = 0
    for X_batch, y_batch in dataloader:
      X_batch = X_batch.to(device)
      y_batch = y_batch.to(device)
      if action == "train":
        model.train()
        optimizer.zero_grad()
        logits = model(X_batch)
      else:
        model.eval()
        with torch.no_grad():
          logits = model(X_batch)
      y_pred = logits.argmax(-1)
      correct_preds = (y_batch == y_pred).sum()
      epoch_correct += correct_preds.item()
      epoch_all += y_batch.shape[0]
      if action == "train":
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()
    if action == "train":
      print(f"Epoch №{epoch + 1}")
    print(f"Loader: {action}, accuracy: {epoch_correct/epoch_all}")
    accuracy[action].append(epoch_correct/epoch_all)

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch №1
Loader: train, accuracy: 0.32292


  1%|          | 1/100 [00:23<38:47, 23.51s/it]

Loader: test, accuracy: 0.41654
Epoch №2
Loader: train, accuracy: 0.44932


  2%|▏         | 2/100 [00:46<38:17, 23.44s/it]

...

Loader: test, accuracy: 0.86558
Epoch №99
Loader: train, accuracy: 0.84598


 99%|█████████▉| 99/100 [38:17<00:23, 23.64s/it]

Loader: test, accuracy: 0.85734
Epoch №100
Loader: train, accuracy: 0.84482


100%|██████████| 100/100 [38:42<00:00, 23.22s/it]

Loader: test, accuracy: 0.85814


In [25]:
torch.save(model.state_dict(), 'CIFAR10_v1_100_EPOCHS.pth')